In [65]:
import pandas as pd

df = pd.read_csv("statswithballsstrikes.txt", sep=' ', header=None) 
  
# display DataFrame 
for column in df.columns:
    df[column] = df[column].str.extract('(\d+)', expand=False)
df['WE'] = df[9].astype(float) / df[10].astype(float)
df = df.drop([9,10], axis=1)
mapping = {df.columns[0]:'Inning', df.columns[1]:'Team', df.columns[2]:'Outs', df.columns[3]:'RO1', 
           df.columns[4]:'RO2', df.columns[5]:'RO3', df.columns[6]:'Run Diff', df.columns[7]:'Balls',
           df.columns[8]: 'Strikes'}
df = df.rename(columns=mapping)
df.to_csv('WE_matrix.csv')

In [37]:
def true_call(plate_x, plate_z, sz_top, sz_bot):
    if ((plate_x > -0.83) & (plate_x < 0.83)) & ((plate_z > sz_bot) & (plate_z < sz_top)):
        return 1
    else:
        return 0

In [50]:
data2023 = pd.read_csv('2023_mlb_statcast.csv')
data2023 = data2023.drop(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher',
       'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'game_year', 'pfx_x',
       'pfx_z', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom', 'babip_value', 'iso_value',
       'launch_speed_angle', 'at_bat_number', 'pitch_number', 'pitch_name',
       'if_fielding_alignment', 'of_fielding_alignment', 'spin_axis'], axis=1)
data2023 = data2023[data2023['events'].isin(['strikeout'])]
data2023 = data2023[data2023['description'].isin(['called_strike'])]
data2023 = data2023.reset_index()
data2023['Challenge'] = data2023.apply(lambda x: true_call(x.plate_x, x.plate_z, x.sz_top, x.sz_bot), axis=1)

In [57]:
successful_chall = data2023[data2023['Challenge'].isin([1])]
successful_chall = successful_chall.reset_index()
for c in ['on_1b', 'on_2b', 'on_3b']:
        successful_chall[c] = successful_chall[c].fillna(0)
successful_chall.loc[successful_chall['on_1b'] > 0, 'on_1b'] = 1
successful_chall.loc[successful_chall['on_2b'] > 0, 'on_2b'] = 1
successful_chall.loc[successful_chall['on_3b'] > 0, 'on_3b'] = 1
successful_chall.loc[successful_chall['inning_topbot'] == 'Top', 'inning_topbot'] = 0
successful_chall.loc[successful_chall['inning_topbot'] == 'Bot', 'inning_topbot'] = 1
#7122 successful challenges

      level_0   index  Unnamed: 0     events    description  balls  strikes  \
0           0      17          17  strikeout  called_strike      2        2   
1           1     291         291  strikeout  called_strike      2        2   
2           2     462         462  strikeout  called_strike      2        2   
3           5     694         694  strikeout  called_strike      2        2   
4           6     750         750  strikeout  called_strike      0        2   
...       ...     ...         ...        ...            ...    ...      ...   
7117     9626  720372      720372  strikeout  called_strike      1        2   
7118     9627  720398      720398  strikeout  called_strike      0        2   
7119     9628  720407      720407  strikeout  called_strike      2        2   
7120     9629  720420      720420  strikeout  called_strike      0        2   
7121     9630  720644      720644  strikeout  called_strike      3        2   

      plate_x  plate_z  on_3b  ...  away_score  bat

In [68]:
def init_call_we(x):
    # inning, team, outs, ro1, ro2, ro3, run_diff, balls, strikes
    df = pd.read_csv('WE_matrix.csv')
    inning = x.loc['inning']
    team = x.loc['inning_topbot']
    outs = x.loc['outs_when_up'] + 1
    ro1 = x.loc['on_1b']
    ro2 = x.loc['on_2b']
    ro3 = x.loc['on_3b']
    if outs == 3:
        outs = 0
        if team == 0:
            team = 1
        else:
            team = 0
        ro1 = 0
        ro2 = 0
        ro3 = 0
    run_diff_top = x.loc['post_away_score'] - x.loc['post_home_score']
    run_diff_bot = x.loc['post_home_score'] - x.loc['post_away_score']
    if team == 0:
        run_diff = run_diff_top
    else:
        run_diff = run_diff_bot
    balls = 0
    strikes = 0
    
    dfc = df[df['Inning'].isin([inning])]
    dfc = dfc[dfc['Team'].isin([team])]
    dfc = dfc[dfc['Outs'].isin([outs])]
    dfc = dfc[dfc['RO1'].isin([ro1])]
    dfc = dfc[dfc['RO2'].isin([ro2])]
    dfc = dfc[dfc['RO3'].isin([ro3])]
    dfc = dfc[dfc['Run Diff'].isin([run_diff])]
    dfc = dfc[dfc['Balls'].isin([balls])]
    dfc = dfc[dfc['Strikes'].isin([strikes])]
    return dfc['WE']

In [69]:
successful_chall['WE_initial'] = successful_chall.apply(init_call_we, axis=1)
print(successful_chall)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
def post_chall_we(x):
    balls = x.loc['balls']
    strikes = x.loc['strikes']

In [ ]:
def challenge_value(x):
    